# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [54]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../Data/Part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [55]:
# 分析缺值
print(df.shape)
df.info()

(891, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [50]:
# 發現 Age/Cabin/Embarked 都不足891列
# 針對 Age補缺值 中位數
df['Age'] = df['Age'].fillna(value=df['Age'].median())

In [51]:
# 針對 Cabin補缺值 
for s in np.unique(df['Cabin'].astype(str)):    
    print("Cabin[{}] ={}".format(s,sum(df['Cabin'] == s)))    
df['Cabin'] = df['Cabin'].fillna(value='Unknown')

Cabin[A10] =1
Cabin[A14] =1
Cabin[A16] =1
Cabin[A19] =1
Cabin[A20] =1
Cabin[A23] =1
Cabin[A24] =1
Cabin[A26] =1
Cabin[A31] =1
Cabin[A32] =1
Cabin[A34] =1
Cabin[A36] =1
Cabin[A5] =1
Cabin[A6] =1
Cabin[A7] =1
Cabin[B101] =1
Cabin[B102] =1
Cabin[B18] =2
Cabin[B19] =1
Cabin[B20] =2
Cabin[B22] =2
Cabin[B28] =2
Cabin[B3] =1
Cabin[B30] =1
Cabin[B35] =2
Cabin[B37] =1
Cabin[B38] =1
Cabin[B39] =1
Cabin[B4] =1
Cabin[B41] =1
Cabin[B42] =1
Cabin[B49] =2
Cabin[B5] =2
Cabin[B50] =1
Cabin[B51 B53 B55] =2
Cabin[B57 B59 B63 B66] =2
Cabin[B58 B60] =2
Cabin[B69] =1
Cabin[B71] =1
Cabin[B73] =1
Cabin[B77] =2
Cabin[B78] =1
Cabin[B79] =1
Cabin[B80] =1
Cabin[B82 B84] =1
Cabin[B86] =1
Cabin[B94] =1
Cabin[B96 B98] =4
Cabin[C101] =1
Cabin[C103] =1
Cabin[C104] =1
Cabin[C106] =1
Cabin[C110] =1
Cabin[C111] =1
Cabin[C118] =1
Cabin[C123] =2
Cabin[C124] =2
Cabin[C125] =2
Cabin[C126] =2
Cabin[C128] =1
Cabin[C148] =1
Cabin[C2] =2
Cabin[C22 C26] =3
Cabin[C23 C25 C27] =4
Cabin[C30] =1
Cabin[C32] =1
Cabin[C45] =1
Cabin[C46]

In [52]:
# 針對 Embarked 補缺值 
for s in np.unique(df['Embarked'].astype(str)):    
    print("Embarked[{}] ={}".format(s,sum(df['Embarked'] == s)))
df['Embarked'] = df['Embarked'].fillna('Unknown')

df.info()

Embarked[C] =168
Embarked[Q] =77
Embarked[S] =644
Embarked[Unknown] =2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       891 non-null object
Embarked    891 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [56]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
# 船票價(Fare)對靠岸口(Embarked)做群聚編碼
"""
Your Code Here
"""
mean_df = df.groupby(['Embarked'])['Fare'].mean().reset_index()
mode_df = df.groupby(['Embarked'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Embarked'])['Fare'].median().reset_index()
max_df = df.groupby(['Embarked'])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Embarked'])
temp = pd.merge(temp, median_df, how='left', on=['Embarked'])
temp = pd.merge(temp, max_df, how='left', on=['Embarked'])
temp.columns = ['Embarked', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']
temp

,Embarked,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,C,59.954144,7.2292,29.70,512.3292
1,Q,13.276030,7.7500,7.75,90.0000
2,S,27.079812,8.0500,13.00,263.0000


In [57]:
df = pd.merge(df, temp, how='left', on=['Embarked'])
df = df.drop(['Embarked'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,27.079812,8.0500,13.0,263.0000
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,59.954144,7.2292,29.7,512.3292
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,27.079812,8.0500,13.0,263.0000
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,27.079812,8.0500,13.0,263.0000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,27.079812,8.0500,13.0,263.0000


In [58]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max']



,Pclass,Age,SibSp,Parch,Fare,Fare_Mean,Fare_Mode,Fare_Median,Fare_Max
0,3,22.0,1,0,7.2500,27.079812,8.0500,13.0,263.0000
1,1,38.0,1,0,71.2833,59.954144,7.2292,29.7,512.3292
2,3,26.0,0,0,7.9250,27.079812,8.0500,13.0,263.0000
3,1,35.0,1,0,53.1000,27.079812,8.0500,13.0,263.0000
4,3,35.0,0,0,8.0500,27.079812,8.0500,13.0,263.0000


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [68]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_minus = df.drop(['Fare_Mean', 'Fare_Mode', 'Fare_Median', 'Fare_Max'] , axis=1)

train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression(solver='lbfgs')
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7004864906462793

In [67]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7027589317916567